In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import *
#create_engine, func

from flask import Flask, jsonify


In [2]:
"""
Now that you have completed your initial analysis, design a Flask API based on the queries that you have just developed.
Use Flask to create your routes.

Routes:
/
    -Home page.
    -List all routes that are available.
"""
from flask import Flask, jsonify

engine = create_engine("sqlite:///hawaii.sqlite")

Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement

app = Flask(__name__)

@app.route("/")
def welcome():   
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
#        f"/api/v1.0/<start>"
#        f"/api/v1.0/<start>/<end>"
        
    )

In [3]:
"""
/api/v1.0/precipitation
    -Convert the query results to a dictionary using date as the key and prcp as the value.
    -Return the JSON representation of your dictionary.
"""
@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)

    results = session.query(Measurement.date, Measurement.prcp).all()

    session.close()
    
    prcp_list = []
    for date, prcp in results:
        prcp_dict = {}
        prcp_dict["date"] = date
        prcp_dict["prcp"] = prcp        
        prcp_list.append(prcp_dict)

    return jsonify(prcp_list)

In [4]:
"""
/api/v1.0/stations
    -Return a JSON list of stations from the dataset.
"""
@app.route("/api/v1.0/stations")
def stations():    
    session = Session(engine)
    stations = session.query(Station.station,Station.name, Station.latitude, Station.longitude, Station.elevation).all()
    session.close()

    all_stations = list(np.ravel(stations))

    return jsonify(stations)

In [6]:
"""
/api/v1.0/tobs
    -Query the dates and temperature observations of the most active station for the last year of data.
    -Return a JSON list of temperature observations (TOBS) for the previous year.
"""
@app.route("/api/v1.0/tobs")
def tobs():
    session=Session(engine)
    tobs=session.query(Measurement.station,Measurement.date, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').filter(Measurement.date >= '2016-08-23').all()
    
    session.close()
    
    return jsonify(tobs)

In [13]:

session = Session(engine)

results = session.query(Measurement.date, Measurement.tobs).\
    order_by(Measurement.date.desc()).all()

session.close()
    
tobs_list = []

for date, tobs in results:
    tobs_dict = {}
    tobs_dict["Date"] = date
    tobs_dict["Tobs"] = tobs
    tobs_list.append(tobs_dict)

tobs_list

        
#        return jsonify(tobs_list)

[{'Date': '2017-08-23', 'Tobs': 81.0},
 {'Date': '2017-08-23', 'Tobs': 82.0},
 {'Date': '2017-08-23', 'Tobs': 82.0},
 {'Date': '2017-08-23', 'Tobs': 76.0},
 {'Date': '2017-08-22', 'Tobs': 82.0},
 {'Date': '2017-08-22', 'Tobs': 82.0},
 {'Date': '2017-08-22', 'Tobs': 76.0},
 {'Date': '2017-08-21', 'Tobs': 81.0},
 {'Date': '2017-08-21', 'Tobs': 79.0},
 {'Date': '2017-08-21', 'Tobs': 82.0},
 {'Date': '2017-08-21', 'Tobs': 76.0},
 {'Date': '2017-08-20', 'Tobs': 81.0},
 {'Date': '2017-08-20', 'Tobs': 80.0},
 {'Date': '2017-08-20', 'Tobs': 78.0},
 {'Date': '2017-08-19', 'Tobs': 79.0},
 {'Date': '2017-08-19', 'Tobs': 80.0},
 {'Date': '2017-08-19', 'Tobs': 85.0},
 {'Date': '2017-08-19', 'Tobs': 71.0},
 {'Date': '2017-08-18', 'Tobs': 80.0},
 {'Date': '2017-08-18', 'Tobs': 77.0},
 {'Date': '2017-08-18', 'Tobs': 79.0},
 {'Date': '2017-08-18', 'Tobs': 76.0},
 {'Date': '2017-08-17', 'Tobs': 78.0},
 {'Date': '2017-08-17', 'Tobs': 83.0},
 {'Date': '2017-08-17', 'Tobs': 76.0},
 {'Date': '2017-08-17', '

In [ ]:
session=Session(engine)
min_all = session.query(Measurement.date, func.min(Measurement.tobs)).\
    group_by(Measurement.date).all()

max_all = session.query(Measurement.date, func.max(Measurement.tobs)).\
    group_by(Measurement.date).all()

avg_all = session.query(Measurement.date, func.avg(Measurement.tobs)).\
    group_by(Measurement.date).all()

avg_all

In [14]:
"""
/api/v1.0/<start> 
    -Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start 
     or start-end range.
    -When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
    -When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and 
     end date inclusive.
"""
@app.route("/api/v1.0/<start>")
def start(start):

    canonicalized = start.replace(" ").lower()
    for x in tobs_list:
        search_term = x["Date"].replace(" ").lower()

        if search_term == canonicalized:
            
            
            
            return jsonify(x)

    return jsonify({"error": f"Character with start date {start} not found."}), 404


In [7]:
if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\mokae\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
